# Sonda Kelvina

In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import uncertainties
from uncertainties import ufloat
from uncertainties.umath import *
from IPython.display import display, Math, Latex

In [114]:
df_CPD_Au = pd.read_csv("Wyniki pomiarow CPD dla Au.dat", skipfooter=31, usecols=range(10), engine='python')
df_CPD_X1 = pd.read_csv("Wyniki pomiarow CPD dla probki X1.dat", skipfooter=31, usecols=range(10), engine='python')
df_CPD_X2 = pd.read_csv("Wyniki pomiarow CPD dla probki X2.dat", skipfooter=31, usecols=range(10), engine='python')
df_CPD_X2

,Point,WF (mV),WFRA (mV),WFDel (mV),Std WF,GD (au),Std GD,Z Height (um),User,Time(Secs)
0,0,-514.9,-514.9,0.0,0.0,299.6,0.0,0.0,1.6,0.000
1,1,-517.6,-516.3,-2.7,0.0,299.8,0.0,0.0,1.4,0.561
2,2,-518.9,-516.7,-3.9,0.0,300.0,0.0,0.0,1.4,1.139
3,3,-510.3,-515.6,4.6,2.0,299.9,0.2,0.0,0.9,1.685
4,4,-514.3,-514.5,0.6,3.8,299.6,0.2,0.0,1.2,2.246
...,...,...,...,...,...,...,...,...,...,...
1496,1496,-517.2,-517.1,-2.3,2.9,294.5,1.5,0.0,0.5,863.803
1497,1497,-517.0,-517.1,-2.1,2.9,294.4,1.5,0.0,1.3,864.411
1498,1498,-510.5,-514.9,4.5,2.9,294.6,1.5,0.0,1.4,865.004
1499,1499,-515.1,-514.2,-0.2,2.9,294.3,1.5,0.0,1.3,865.581


In [111]:
CPD_Au = ufloat(df_CPD_Au['CPD (mV)'].mean(), df_CPD_Au['CPD (mV)'].sem()) # mV
CPD_X1 = ufloat(df_CPD_X1['CPD (mV)'].mean(), df_CPD_X1['CPD (mV)'].sem()) # mV
CPD_X2 = ufloat(df_CPD_X2['CPD (mV)'].mean(), df_CPD_X2['CPD (mV)'].sem()) # mV

graph = sns.scatterplot(x="Point", y="CPD (mV)", data=df_CPD_Au, s=2, label="Pomiary")
graph.axhline(CPD_Au.n, color='r', linestyle='-', label='$\overline{CPD}_{Au}}$')
graph.fill_between(df_CPD_Au["Point"], CPD_Au.n-CPD_Au.s, CPD_Au.n+CPD_Au.s, color='r', alpha=0.25, label='$\hat{\sigma}_{\overline{CPD}_{Au}}$')
graph.set_xlim(df_CPD_Au["Point"].min(), df_CPD_Au["Point"].max())
graph.legend()

display(Latex(f"$\overline{{CPD}}_{{Au}} =$ {CPD_Au:.2uP} mV"))
display(Latex(f"$\overline{{CPD}}_{{X1}} =$ {CPD_X1:.2uP} mV"))
display(Latex(f"$\overline{{CPD}}_{{X2}} =$ {CPD_X2:.2uP} mV"))

KeyError: 'CPD (mV)'

In [109]:
WF_Au = ufloat(4800, 3) # meV
# e = 1.602176634e-19 # C
e = 1 # e
WF_tip = WF_Au - 1*e * CPD_Au
display(Latex(f"$WF_{{tip}} =$ {WF_tip:.2uP} meV"))

<IPython.core.display.Latex object>

Zaokrąglanie liczb i niepewności zgodnie z wytycznymi Particle Data Group https://pdg.lbl.gov/2010/reviews/rpp2010-rev-rpp-intro.pdf